In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv

In [4]:
load_dotenv(dotenv_path=find_dotenv())
from financial.model import Model
import financial.data as fd
from financial.io.file.cache import FileCache
from financial.io.cache import NoUpdateStrategy

In [5]:
print(os.environ["DATA"])
print(os.environ["CACHE"])
ds = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=os.environ["CACHE"]+"/", update_strategy=NoUpdateStrategy()))

/home/manidmt/TFG/OTRI/data
/home/manidmt/TFG/OTRI/cache


In [6]:
data = ds.get_data("M2NS")
print(data)

DATE
1959-01-01      289.8
1959-02-01      287.7
1959-03-01      287.9
1959-04-01      290.2
1959-05-01      290.2
               ...   
2025-01-01    21584.4
2025-02-01    21558.1
2025-03-01    21868.4
2025-04-01    21967.5
2025-05-01    21845.6
Name: M2NS, Length: 797, dtype: float64


In [7]:
import pandas as pd
import shutil

def clean_prediction_pickles(path_dir: str):
    for filename in os.listdir(path_dir):
        # Solo archivos que sean predicciones
        if not (filename.startswith("model-momentum-") and filename.endswith("@pred")):
            continue

        full_path = os.path.join(path_dir, filename)
        backup_path = full_path + "1"  # e.g., @pred1

        if not os.path.isfile(full_path):
            continue

        # Si ya hay backup, evitar sobrescribirlo
        if not os.path.exists(backup_path):
            shutil.copy2(full_path, backup_path)
            print(f"📦 Backup creado: {os.path.basename(backup_path)}")
        else:
            print(f"⚠️ Backup ya existe: {os.path.basename(backup_path)}")

        try:
            obj = pd.read_pickle(full_path)

            if isinstance(obj, pd.Series):
                changed = False

                if obj.index.name is not None:
                    obj.index.name = None
                    changed = True

                if obj.name is not None:
                    obj.name = None
                    changed = True

                if changed:
                    pd.to_pickle(obj, full_path)
                    print(f"✅ Limpiado: {filename}")
                else:
                    print(f"✅ Ya limpio: {filename}")
            else:
                print(f"ℹ️ No es Series (omitido): {filename} [{type(obj)}]")

        except Exception as e:
            print(f"❌ Error al procesar {filename}: {e}")

#clean_prediction_pickles(path_dir=os.environ["CACHE"]+"/")

In [8]:
def clean_prediction_pickles2(path_dir: str):
    for filename in os.listdir(path_dir):
        if not (filename.startswith("model-momentum-") and filename.endswith("@pred")):
            continue

        full_path = os.path.join(path_dir, filename)
        if not os.path.isfile(full_path):
            continue

        try:
            obj = pd.read_pickle(full_path)

            if isinstance(obj, pd.Series):
                changed = False

                # Asegura que todos los valores sean escalares
                if isinstance(obj.iloc[0], pd.Series):
                    obj = obj.apply(lambda x: x.iloc[0] if isinstance(x, pd.Series) else x)
                    changed = True

                # Elimina nombre de índice
                if obj.index.name is not None:
                    obj.index.name = None
                    changed = True

                # Elimina nombre de la serie
                if obj.name is not None:
                    obj.name = None
                    changed = True

                if changed:
                    pd.to_pickle(obj, full_path)
                    print(f"✅ Limpiado: {filename}")
                else:
                    print(f"✅ Ya limpio: {filename}")
            else:
                print(f"ℹ️ No es Series (omitido): {filename} [{type(obj)}]")

        except Exception as e:
            print(f"❌ Error al procesar {filename}: {e}")

#clean_prediction_pickles2(path_dir=os.environ["CACHE"]+"/")

In [9]:

def check_nested_series(path_dir: str):
    all_good = True

    for filename in os.listdir(path_dir):
        if not (filename.startswith("model-momentum-") and filename.endswith("@pred")):
            continue

        full_path = os.path.join(path_dir, filename)

        if not os.path.isfile(full_path):
            continue

        try:
            obj = pd.read_pickle(full_path)

            if not isinstance(obj, pd.Series):
                print(f"❌ {filename} no es una pd.Series → {type(obj)}")
                all_good = False
                continue

            # Comprueba si el primer valor es una Serie (posible Series anidada)
            first_val = obj.iloc[0]

            if isinstance(first_val, pd.Series):
                print(f"💣 {filename} contiene Series anidadas: type(obj.iloc[0]) = {type(first_val)}")
                all_good = False
            else:
                print(f"✅ {filename} OK — type(obj.iloc[0]) = {type(first_val)}")

        except Exception as e:
            print(f"❌ Error procesando {filename}: {e}")
            all_good = False

    if all_good:
        print("\n🎉 TODOS LOS PICKLES ESTÁN CORRECTOS — sin Series anidadas ✅")
    else:
        print("\n⚠️ Hay archivos con problemas. Revisa los mensajes arriba.")

#check_nested_series(path_dir=os.environ["CACHE"]+"/")

In [10]:
from financial.strategies.technical.indicator import Wrapper
import financial.data as fd
import os

ticker = "GOOG"
name = "cnn_GOOG_2025_single"

series = Wrapper().set_parameters({
    "ticker": f"model-momentum-{name}@pred",
    "default": f"model-momentum-{name}@pred"
}).get_data(ds, "GOOG")

print(type(series))
print(series.index.name)  # <- Debe ser None
print(series.name)        # <- También None
print(series.at["2020-01-10"])

<class 'pandas.core.series.Series'>
None
GOOG_Wrapper
68.09816717037232


In [11]:
series = fd.Default("model-momentum-rnn_BAM_2025_single@pred", "model-momentum-rnn_BAM_2025_single@pred").get_data(ds, "2025-06-27", "2025-06-28")
print(type(series), series)

<class 'pandas.core.series.Series'> 2025-06-27    55.89905
dtype: float64


In [12]:
import pickle
import pandas as pd

cache_path = os.environ["CACHE"]
simulation_universe = ['BLK', '^IBEX', 'BAM', '^IXIC', 'GOOG', 'TSLA']
for ticker in simulation_universe:
    model_name = f"rnn_{ticker}_2025_single"
    prediction_path = os.path.join(cache_path, f"model-momentum-{model_name}@pred")
    series = pd.read_pickle(prediction_path)
    print(type(series))
    print(series.head())
    print(series.index)
    print(series.name)
    with open(prediction_path, 'rb') as file:
                prediction_series = pickle.load(file)
                print(type(prediction_series))
                print(f"{ticker}\n")
                print("\n📊 **Prediction Series")
                #print(prediction_series.apply(lambda x: f"{x:.15f}") + "%")
                print(prediction_series)

<class 'pandas.core.series.Series'>
2000-03-09    11.198337
2000-03-10    10.893617
2000-03-13    10.931707
2000-03-14    10.969798
2000-03-15    10.893618
dtype: float64
DatetimeIndex(['2000-03-09', '2000-03-10', '2000-03-13', '2000-03-14',
               '2000-03-15', '2000-03-16', '2000-03-17', '2000-03-20',
               '2000-03-21', '2000-03-22',
               ...
               '2025-06-16', '2025-06-17', '2025-06-18', '2025-06-20',
               '2025-06-23', '2025-06-24', '2025-06-25', '2025-06-26',
               '2025-06-27', '2025-06-30'],
              dtype='datetime64[ns]', length=6365, freq=None)
None
<class 'pandas.core.series.Series'>
BLK


📊 **Prediction Series
2000-03-09     11.198337
2000-03-10     10.893617
2000-03-13     10.931707
2000-03-14     10.969798
2000-03-15     10.893618
                 ...    
2025-06-24    966.407667
2025-06-25    984.608511
2025-06-26    974.534794
2025-06-27    979.591662
2025-06-30    983.173641
Length: 6365, dtype: float64
<cla

In [13]:
path=os.path.join(cache_path, "model-momentum-exponential-^GSPC@r2")
serie_buena = pd.read_pickle(path)
print("✅ SERIE BUENA")
print(type(serie_buena))
print(serie_buena.index)
print("index.name:", serie_buena.index.name)
print("series.name:", serie_buena.name)
print("first value:", serie_buena.dropna().iloc[0])

✅ SERIE BUENA
<class 'pandas.core.series.Series'>
DatetimeIndex(['1990-09-18', '1990-09-19', '1990-09-20', '1990-09-21',
               '1990-09-24', '1990-09-25', '1990-09-26', '1990-09-27',
               '1990-09-28', '1990-10-01',
               ...
               '2024-12-17', '2024-12-18', '2024-12-19', '2024-12-20',
               '2024-12-23', '2024-12-24', '2024-12-26', '2024-12-27',
               '2024-12-30', '2024-12-31'],
              dtype='datetime64[ns]', length=8637, freq=None)
index.name: None
series.name: None
first value: 0.6098779930681638


In [14]:
path=os.path.join(cache_path, "model-momentum-rnn_^IXIC_2025_single@pred")
serie_mala = pd.read_pickle(path)
print("❌ SERIE MALA")
print(type(serie_mala))
print(repr(series))
print(serie_mala.index)
print("index.name:", serie_mala.index.name)
print("series.name:", serie_mala.name)
print("first value:", serie_mala.iloc[0])
print("first value type:", type(serie_mala.iloc[0]))

❌ SERIE MALA
<class 'pandas.core.series.Series'>
2010-12-03      1.682006
2010-12-06      1.651370
2010-12-07      1.690243
2010-12-08      1.670021
2010-12-09      2.013952
                 ...    
2025-06-24    343.710868
2025-06-25    367.581576
2025-06-26    361.506410
2025-06-27    363.051685
2025-06-30    350.918314
Length: 3664, dtype: float64
DatetimeIndex(['1990-06-08', '1990-06-11', '1990-06-12', '1990-06-13',
               '1990-06-14', '1990-06-15', '1990-06-18', '1990-06-19',
               '1990-06-20', '1990-06-21',
               ...
               '2025-06-16', '2025-06-17', '2025-06-18', '2025-06-20',
               '2025-06-23', '2025-06-24', '2025-06-25', '2025-06-26',
               '2025-06-27', '2025-06-30'],
              dtype='datetime64[ns]', length=8830, freq=None)
index.name: None
series.name: None
first value: 462.18306554227553
first value type: <class 'numpy.float64'>


In [15]:
from financial.momentum.indicators.exponentialIndicator import ExponentialRegressionIndicator

serie_exp = ExponentialRegressionIndicator()
serie_exp = serie_exp.get_data(ds, "GSPC")

print("✅ WRAPPER BUENO")
print(type(serie_exp))
print(serie_exp.index)
print("index.name:", serie_exp.index.name)
print("series.name:", serie_exp.name)
print("value @ 2020-01-03:", serie_exp.get("2020-01-03"))
print("type of value:", type(serie_exp.get("2020-01-03")))

✅ WRAPPER BUENO
<class 'pandas.core.series.Series'>
DatetimeIndex(['1990-09-18', '1990-09-19', '1990-09-20', '1990-09-21',
               '1990-09-24', '1990-09-25', '1990-09-26', '1990-09-27',
               '1990-09-28', '1990-10-01',
               ...
               '2024-12-17', '2024-12-18', '2024-12-19', '2024-12-20',
               '2024-12-23', '2024-12-24', '2024-12-26', '2024-12-27',
               '2024-12-30', '2024-12-31'],
              dtype='datetime64[ns]', length=8637, freq=None)
index.name: None
series.name: GSPC_ExponentialRegressionIndicator
value @ 2020-01-03: 0.0010041803838931815
type of value: <class 'numpy.float64'>


In [16]:
from financial.momentum.indicators.kerasIndicator import KerasIndicator
simulation_indicator = KerasIndicator("rnn")
serie_keras = simulation_indicator.get_data(ds, "^IXIC")

print("❌ WRAPPER MALO")
print(type(serie_keras))
print(repr(serie_keras))
print(serie_keras.index)
print("index.name:", serie_keras.index.name)
print("series.name:", serie_keras.name)
print("value @ 2019-01-31:", serie_keras.get("2019-01-31"))
print("type of value:", type(serie_keras.get("2019-01-31")))

❌ WRAPPER MALO
<class 'pandas.core.series.Series'>
1990-06-08      462.183066
1990-06-11      470.755638
1990-06-12      472.719342
1990-06-13      471.908730
1990-06-14      468.683542
                  ...     
2025-06-24    19124.782515
2025-06-25    19587.222992
2025-06-26    19639.661734
2025-06-27    19694.820730
2025-06-30    19866.177733
Name: ^IXIC_KerasIndicator, Length: 8830, dtype: float64
DatetimeIndex(['1990-06-08', '1990-06-11', '1990-06-12', '1990-06-13',
               '1990-06-14', '1990-06-15', '1990-06-18', '1990-06-19',
               '1990-06-20', '1990-06-21',
               ...
               '2025-06-16', '2025-06-17', '2025-06-18', '2025-06-20',
               '2025-06-23', '2025-06-24', '2025-06-25', '2025-06-26',
               '2025-06-27', '2025-06-30'],
              dtype='datetime64[ns]', length=8830, freq=None)
index.name: None
series.name: ^IXIC_KerasIndicator
value @ 2019-01-31: 7313.958716217321
type of value: <class 'numpy.float64'>


In [17]:
model = Model.from_file("rnn_BLK_2025_single", path=os.environ["MODEL"], model_type="financial.momentum.models.kerasAdvanced.RecurrentModel")
print(type(model))  # debería dar RecurrentModel
model.model.compile(
    loss="mean_squared_error",
    optimizer="adam",
    metrics=["mae"]
)
predictions = model.get_data(ds, start_index="2010-01-01", end_index="2023-12-31")
print(predictions)

2025-07-25 16:51:48.939210: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-07-25 16:51:48.939234: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-07-25 16:51:48.939239: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: manidmt-Katana-17-B13VGK
2025-07-25 16:51:48.939243: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] hostname: manidmt-Katana-17-B13VGK
2025-07-25 16:51:48.939433: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:197] libcuda reported version is: 570.169.0
2025-07-25 16:51:48.939448: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:201] kernel re

<class 'financial.momentum.models.kerasAdvanced.RecurrentModel'>
108/108 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step
Date
2010-05-13   -0.008589
2010-05-14   -0.008589
2010-05-17   -0.008589
2010-05-18   -0.008589
2010-05-19   -0.008589
                ...   
2023-12-22   -0.008591
2023-12-26   -0.008591
2023-12-27   -0.008591
2023-12-28   -0.008591
2023-12-29   -0.008591
Length: 3432, dtype: float32


In [18]:
model_rforest = Model.from_file('transformer_BLK_2025_single', path = os.environ["MODEL"])
print(type(model))
predictions_rforest = model_rforest.get_data(ds, start_index="2010-01-01", end_index="2023-12-31")
print(predictions_rforest)

<class 'financial.momentum.models.kerasAdvanced.RecurrentModel'>
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
Date
2010-05-13   -0.008092
2010-05-14   -0.008092
2010-05-17   -0.008092
2010-05-18   -0.008092
2010-05-19   -0.008092
                ...   
2023-12-22   -0.008092
2023-12-26   -0.008092
2023-12-27   -0.008092
2023-12-28   -0.008092
2023-12-29   -0.008092
Length: 3432, dtype: float32


In [19]:
from keras.models import load_model
from lxml import etree
import json
from financial.momentum.models.kerasAdvanced import RecurrentModel
file = "rnn_GOOG_2025_single"
path = os.environ["MODEL"]
xml_path = os.path.join(path, file + ".xml")
base_path = os.path.join(path, file)

# 1. Cargar metadatos
xml = Model.metadata_from_xml_file(base_path)
model_id, sources, target = xml

# 2. Cargar hiperparámetros
with open(base_path + ".hyperparameters.json") as f:
    hyperparameters = json.load(f)

# 3. Cargar modelo Keras desde archivo (.keras o .h5)
model_path = base_path + ".keras"
if not os.path.exists(model_path):
    model_path = base_path + ".h5"
model = load_model(model_path, compile=False)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

# 4. Devolver una instancia real de RecurrentModel
final_model=RecurrentModel(model_id, sources, target, model=model, hyperparameters=hyperparameters)

final_model.model.summary()
weights = model.get_weights()
import numpy as np
print("Número de capas con pesos:", len(weights))
print("STD de las últimas capas:", np.std(weights[-1]))

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_30 (SimpleRNN)       │ (None, 90, 64)         │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_31 (SimpleRNN)       │ (None, 90, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_32 (SimpleRNN)       │ (None, 1)              │           130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,060 (113.52 KB)

 Trainable params: 29,060 (113.52 KB)

 Non-trainable params: 0 (0.00 B)

Número de capas con pesos: 11
STD de las últimas capas: 0.0


In [21]:
X = final_model.get_input_data(ds, start_index="2010-01-01", end_index="2023-12-31")
X_reshaped = final_model.reshape_input(X)
print("Input shape:", X.shape)
print("Reshaped input shape:", X_reshaped.shape)

print("Varianza de entrada:", np.var(X_reshaped, axis=(1, 2)))
print("Valores únicos en salida:", np.unique(final_model.predict(X)))

Input shape: (3522, 90)
Reshaped input shape: (3522, 90, 1)
Varianza de entrada: [       nan        nan        nan ... 0.00117106 0.00115942 0.00114931]
111/111 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step
Valores únicos en salida: [-0.01457071 -0.01408962 -0.01405234 ... -0.00041075 -0.00034997
         nan]


In [23]:
file = "cnn_GOOG_2025_single"
path = os.environ["MODEL"]
base_path = os.path.join(path, file)
model_path = base_path + ".keras"
model = load_model(model_path)
model = load_model(model_path, compile=False)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()
weights = model.get_weights()
import numpy as np
print("Número de capas con pesos:", len(weights))
print("STD de las últimas capas:", np.std(weights[-1]))

/home/manidmt/miniconda3/envs/tfg_gpu/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_20 (Conv1D)              │ (None, 90, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_20 (MaxPooling1D) │ (None, 45, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_21 (Conv1D)              │ (None, 45, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_21 (MaxPooling1D) │ (None, 22, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 2816)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │         2,817 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,777 (108.50 KB)

 Trainable params: 27,777 (108.50 KB)

 Non-trainable params: 0 (0.00 B)

Número de capas con pesos: 6
STD de las últimas capas: 0.0


In [25]:
file_format = "h5"
filepath = os.path.join(os.environ["MODEL"], file + "." + file_format)
model=load_model(filepath)
file_format = "xml"
filepath = os.path.join(os.environ["MODEL"], file + "." + file_format)
inputDescriptor = fd.DataDescriptor.from_xml_string(filepath)

ParseError: not well-formed (invalid token): line 1, column 0 (<string>)